# Exercise 2: Advanced Analytics NLP

In [1]:
!pip install spark-nlp==1.7.3

    100% |████████████████████████████████| 72.8MB 500kB/s eta 0:00:01  3% |█▎                              | 2.9MB 19.9MB/s eta 0:00:04    6% |██                              | 4.8MB 20.8MB/s eta 0:00:04    8% |██▋                             | 5.8MB 22.3MB/s eta 0:00:03    14% |████▉                           | 10.9MB 21.5MB/s eta 0:00:03    17% |█████▊                          | 13.0MB 18.8MB/s eta 0:00:04    19% |██████▏                         | 14.0MB 21.2MB/s eta 0:00:03    20% |██████▋                         | 14.9MB 20.6MB/s eta 0:00:03    24% |███████▉                        | 17.8MB 19.3MB/s eta 0:00:03    28% |█████████                       | 20.7MB 21.1MB/s eta 0:00:03    29% |█████████▌                      | 21.7MB 21.8MB/s eta 0:00:03    31% |██████████                      | 22.6MB 19.4MB/s eta 0:00:03    36% |███████████▌                    | 26.2MB 20.6MB/s eta 0:00:03    38% |████████████▍                   | 28.1MB 18.0MB/s eta 0:00:03    39% |████████████▊      

In [2]:
import pandas as pd
pd.set_option('max_colwidth', 800)

# Create a spark context that includes a 3rd party jar for NLP

In [3]:
#jarPath = "spark-nlp-assembly-1.7.3.jar"

from pyspark.sql import SparkSession
# Todo
spark = SparkSession.builder\
        .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:1.8.2")\
        .getOrCreate()

In [4]:
spark

# Read multiple files in a dir as one Dataframe

In [5]:
dataPath = "./data/reddit/*.json"
df = spark.read.json(dataPath)
print(df.count())
df.printSchema()

100
root
 |-- data: struct (nullable = true)
 |    |-- approved_at_utc: string (nullable = true)
 |    |-- approved_by: string (nullable = true)
 |    |-- archived: boolean (nullable = true)
 |    |-- author: string (nullable = true)
 |    |-- author_flair_background_color: string (nullable = true)
 |    |-- author_flair_css_class: string (nullable = true)
 |    |-- author_flair_richtext: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- author_flair_template_id: string (nullable = true)
 |    |-- author_flair_text: string (nullable = true)
 |    |-- author_flair_text_color: string (nullable = true)
 |    |-- author_flair_type: string (nullable = true)
 |    |-- author_fullname: string (nullable = true)
 |    |-- author_patreon_flair: boolean (nullable = true)
 |    |-- banned_at_utc: string (nullable = true)
 |    |-- banned_by: string (nullable = true)
 |    |-- can_gild: boolean (nullable = true)
 |    |-- can_mod_post: boolean (nullable = true)


# Deal with Struct type to query subfields 

In [8]:
title = "data.title"
author = "data.author"

# Todo
dfFiltered = df.select(title, author)

In [10]:
dfFiltered.limit(5).toPandas()

title  \
0                                                                                                                                                                   Microsoft Corp said it has discovered hacking targeting democratic institutions, think tanks, and non-profit organizations in Europe.   
1                                                                                                                              Deutsche Bank reportedly planned to extend the dates of $340 million in loans to Trump Organization to avoid a potential nightmare of chasing a sitting president for cash   
2  Iranian "morality police" were forced to fire warning shots when a crowd intervened to prevent them from arresting two women for not wearing a hijab. The incident occurred in Tehran's northeastern Narmak neighbourhood on Friday night, and ended with a mob tearing the door off a police vehicle.   
3         Trump administration 'pushing Saudi nuclear deal' which could benefit company linked to Jared Kushner - Senior Trump administration officials pushed a project to share nuclear power technology with Saudi Arabia over the objections of ethics officials, according to a congressional report   
4                                                                                                                                                                                            NASA Happily Reports the Earth is Greener, With More Trees Than 20 Years Ago–and It's Thanks to China, India   

              author  
0          jaykirsch  
1      canuck_burger  
2  honolulu_oahu_mod  
3             madam1  
4          purplexxx

# Try to implement the equivalent of flatMap in dataframes

In [23]:
import pyspark.sql.functions as F

#dfWordCount = # Todo

dfWordCount = dfFiltered.select(F.explode(F.split("title", "\\s+")).alias("word")).groupBy("word").count().orderBy(F.desc("count"))
dfWordCount.show()

+-----+-----+
| word|count|
+-----+-----+
|   to|   58|
|  the|   46|
|   of|   42|
|   in|   41|
|    a|   25|
|  for|   20|
|  and|   19|
| from|   12|
|   on|   11|
| with|   10|
| over|   10|
|   as|    9|
|  The|    8|
|Saudi|    8|
|   is|    8|
|   be|    8|
|   US|    8|
|Trump|    7|
| have|    7|
|Putin|    7|
+-----+-----+
only showing top 20 rows



# Use an NLP libary to do Part-of-Speech Tagging

In [28]:
from com.johnsnowlabs.nlp.pretrained.pipeline.en import BasicPipeline as bp

In [37]:
#ToDo
dfAnnotated = bp.annotate(dfFiltered, "title")

In [38]:
dfAnnotated.printSchema()

root
 |-- text: string (nullable = true)
 |-- author: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |-- normal: array (nullable = true)
 |    |-- element: struct (contains

## Deal with Map type to query subfields

In [42]:
#dfPos = # Todo
dfPos = dfAnnotated.select("text", "pos.metadata", "pos.result")

In [43]:
dfPos.show(5)

+--------------------+--------------------+--------------------+
|                text|            metadata|              result|
+--------------------+--------------------+--------------------+
|Microsoft Corp sa...|[[word -> Microso...|[NNP, NNP, VBD, P...|
|Deutsche Bank rep...|[[word -> Deutsch...|[NNP, NNP, RB, VB...|
|Iranian "morality...|[[word -> Iranian...|[JJ, NN, NN, VBD,...|
|Trump administrat...|[[word -> Trump],...|[NNP, NN, VBG, NN...|
|NASA Happily Repo...|[[word -> NASA], ...|[NNP, NNP, NNS, D...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [44]:
#dfPos= # Todo
dfPos = dfAnnotated.select(F.explode("pos").alias("pos"))

In [45]:
dfPos.limit(10).toPandas()

pos
0       (pos, 0, 8, NNP, {'word': 'Microsoft'})
1          (pos, 10, 13, NNP, {'word': 'Corp'})
2          (pos, 15, 18, VBD, {'word': 'said'})
3            (pos, 20, 21, PRP, {'word': 'it'})
4           (pos, 23, 25, VBZ, {'word': 'has'})
5    (pos, 27, 36, VBN, {'word': 'discovered'})
6       (pos, 38, 44, VBG, {'word': 'hacking'})
7     (pos, 46, 54, VBG, {'word': 'targeting'})
8     (pos, 56, 65, JJ, {'word': 'democratic'})
9  (pos, 67, 78, NNS, {'word': 'institutions'})

In [46]:
dfPos.printSchema()

root
 |-- pos: struct (nullable = true)
 |    |-- annotatorType: string (nullable = true)
 |    |-- begin: integer (nullable = false)
 |    |-- end: integer (nullable = false)
 |    |-- result: string (nullable = true)
 |    |-- metadata: map (nullable = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)



## Keep only proper nouns NNP or NNPS

In [51]:
nnpFilter = "pos.result = 'NNP' or pos.result = 'NNPS' "
#dfNNP = # Todo
dfNNP = dfPos.where(nnpFilter)

In [52]:
dfNNP.limit(10).toPandas()

pos
0       (pos, 0, 8, NNP, {'word': 'Microsoft'})
1          (pos, 10, 13, NNP, {'word': 'Corp'})
2      (pos, 126, 131, NNP, {'word': 'Europe'})
3        (pos, 0, 7, NNP, {'word': 'Deutsche'})
4           (pos, 9, 12, NNP, {'word': 'Bank'})
5         (pos, 81, 85, NNP, {'word': 'Trump'})
6  (pos, 87, 98, NNP, {'word': 'Organization'})
7      (pos, 175, 180, NNP, {'word': 'Tehran'})
8      (pos, 197, 202, NNP, {'word': 'Narmak'})
9      (pos, 221, 226, NNP, {'word': 'Friday'})

## Extract columns form a map in a col

In [56]:
#dfWordTag = # Todo
dfWordTag = dfNNP.selectExpr("pos.metadata['word'] as word", "pos.result as tag")

In [57]:
dfWordTag.show(4)

+---------+---+
|     word|tag|
+---------+---+
|Microsoft|NNP|
|     Corp|NNP|
|   Europe|NNP|
| Deutsche|NNP|
+---------+---+
only showing top 4 rows



In [59]:
from pyspark.sql.functions import desc
# Todo
dfWordTag.groupBy('word').count().orderBy(desc('count')).show()

+--------+-----+
|    word|count|
+--------+-----+
|      US|   14|
|   Trump|    9|
|   Saudi|    8|
|   Putin|    7|
|  Russia|    6|
|  Europe|    5|
|  Arabia|    5|
|Catholic|    4|
|      UK|    4|
|Vladimir|    4|
|   China|    3|
| Germany|    3|
|    Pope|    3|
|   Egypt|    3|
|   South|    3|
|   House|    3|
|  Church|    3|
|National|    2|
|  Africa|    2|
|   India|    2|
+--------+-----+
only showing top 20 rows

